In [2]:
import os
import matplotlib
import numpy as np
np.random.seed(1234)
import matplotlib.pyplot as plt
import csv
import lasagne
import theano
import theano.tensor as T
from numpy import genfromtxt
from lasagne.layers import batch_norm ,dropout,DenseLayer

conv = lasagne.layers.Conv2DLayer
pool = lasagne.layers.MaxPool2DLayer
NUM_EPOCHS = 500
BATCH_SIZE = 256
LEARNING_RATE = 0.001
DIM = 48
DATA_SIZE = 35887
NUM_CLASSES = 10
FILE_NAME = "fer2013/fer2013.csv"

Using gpu device 0: GeForce GT 740M (CNMeM is disabled, cuDNN 5105)


In [4]:
def build_model(input_width, input_height, output_dim):
    ini = lasagne.init.HeUniform(gain='relu')
    l_in = lasagne.layers.InputLayer(shape=(None, 1, input_width, input_height),)
    
    
    #class_l5 = pool(class_l4, pool_size=(2, 2))
    #class_b1 = lasagne.layers.batch_norm(class_l1) #batchnorm
    #class_d1 = lasagne.layers.DropoutLayer(class_l2) #  dropout
    
    # Classi32fication network
    #stack_1 = batch_norm(ConvLayer(l, num_filters=out_num_filters,filter_size=(3,3), stride=first_stride, nonlinearity=rectify, pad='same', W=lasagne.init.HeNormal(gain='relu'), flip_filters=False))
    class_l1 = batch_norm(conv(
        l_in,
        num_filters=32,pad='same',
        filter_size=(3, 3),
        #filter_size=(5, 5),
        nonlinearity=lasagne.nonlinearities.rectify,
        W=ini,
    ))
    class_d0 = dropout(class_l1,p=0.3)
    
    class_l2 = batch_norm(conv(
        class_d0,
        num_filters=32,pad='same',
        filter_size=(3, 3),
        #filter_size=(5, 5),
        nonlinearity=lasagne.nonlinearities.rectify,
        W=ini,
    ))
    class_d1 = dropout(class_l2,p=0.3)
    
    
    class_l3 = batch_norm(conv(
        class_d1,
        num_filters=32,pad='same',
        filter_size=(3, 3),
        #filter_size=(5, 5),
        nonlinearity=lasagne.nonlinearities.rectify,
        W=ini,
    ))
    class_d2 = dropout(class_l3,p=0.35)
    
    
    class_l4 = batch_norm(conv(
        class_d2,
        num_filters=32,pad='same',
        filter_size=(3, 3),
        #filter_size=(5, 5),
        nonlinearity=lasagne.nonlinearities.rectify,
        W=ini,
    ))
    class_d3 = dropout(class_l4,p=0.2)   
    #class_l5 = pool(class_l4, pool_size=(2, 2))
    
    
    class_l1_dens = batch_norm(DenseLayer(
        class_d3,
        num_units=32,
        nonlinearity=lasagne.nonlinearities.rectify,
        W=ini,
    )) 
    class_d1_dens = dropout(class_l1_dens,0.3)
    
    class_l2_dens = batch_norm(DenseLayer(
        class_d1_dens,
        num_units=32,
        nonlinearity=lasagne.nonlinearities.rectify,
        W=ini,
    )) 
    class_d2_dens = dropout(class_l2_dens,p=0.2)
    
    l_out = DenseLayer(
        class_d2_dens,
        num_units=output_dim,
        nonlinearity=lasagne.nonlinearities.softmax,
        W=ini,
    )
    return l_out

model= build_model(DIM, DIM, NUM_CLASSES)
print("number of parameters in model: %d" % lasagne.layers.count_params(model, trainable=True))

number of parameters in model: 2388970


In [3]:
#Setting up the graph in theano
sym_x = T.tensor4('sym_x') # a symbolic variable, this is now a 4-D tensor.
sym_t = T.ivector('sym_t') # a symbolic variable taking on the value of the target batch.

# Retrieve network output
train_out = lasagne.layers.get_output(model, sym_x, deterministic=False)
eval_out = lasagne.layers.get_output(model, sym_x, deterministic=True)

# Retrieve list of all trainable parameters in the network.
all_params = lasagne.layers.get_all_params(model, trainable=True)

# add weight decay
all_layers = lasagne.layers.get_all_layers(model)
l2_penalty = lasagne.regularization.regularize_layer_params(all_layers, lasagne.regularization.l2) * 0.001


#reg2 = lasagne.regularization.l2(train_out)
#reg = lasagne.regularization.l1( train_out )
cost = lasagne.objectives.categorical_crossentropy(train_out+1e-8, sym_t).mean()

# Let Theano do its magic and get all the gradients we need for training
all_grads = T.grad(cost, all_params)
 
# Set the update function for parameters 
# you might wan't to experiment with more advanded update schemes like rmsprob, adadelta etc.
sh_lr = theano.shared(lasagne.utils.floatX(LEARNING_RATE))

Updates = lasagne.updates.adam(all_grads, all_params, learning_rate=sh_lr)

f_eval = theano.function([sym_x],eval_out, on_unused_input='warn')

f_train = theano.function([sym_x, sym_t],[cost],updates=Updates, on_unused_input='warn')

In [4]:
with np.load('weights.npz') as f:
    param_values = [f['arr_%d' % i] for i in range(len(f.files))]
lasagne.layers.set_all_param_values( model, param_values)

In [21]:
import cv2
import dlib
import matplotlib.pyplot as plt
# (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#fig = plt.figure()
#cv2.destroyAllWindows()
#cap = cv2.VideoCapture(0)
#while(True):
    # Capture frame-by-frame
   # ret, frame = cap.read()
    
#start here
predictor_path = 'shape_predictor_68_face_landmarks.dat'
image_path = 'testImg/test10.jpg'
#os.system('pwd')
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)
# (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
frame = cv2.imread(image_path,cv2.IMREAD_UNCHANGED)
#happy = cv2.imread('emojis/happy.png', cv2.IMREAD_UNCHANGED)
#happy = cv2.cvtColor(happy, cv2.COLOR_BGRA2RGBA)
#cv2.imshow('hi',happy)
#cv2.waitKey(0)
#plt.imshow(happy)
#print len(frame)
img = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

facelocs = detector(img, 3)
faces = np.ndarray(shape=(1, 48*48), dtype=np.uint8)
for box in facelocs:
    face = gray[box.top(): box.bottom(), box.left(): box.right()]
    face = cv2.resize(face, (48, 48), interpolation = cv2.INTER_CUBIC)
    face = np.array(face)
    face = np.reshape(face, 48*48)
    faces = np.vstack((faces, face))
faces = np.delete(faces, 0, 0)
if faces.shape[0]!=0: 
    modes = np.array(['emojis/happy.png','emojis/disgust.png','emojis/fear.png','emojis/happy.png',
                  'emojis/sad.png','emojis/surprised.png','emojis/neutral.png'])
    faces = faces.reshape((faces.shape[0], 1, DIM, DIM))
    #print faces.shape
    net_out = f_eval(faces)   
    preds = np.argmax(net_out, axis=-1)
i=0
for box in facelocs:
    shapes = predictor(img, box)

    emoji = cv2.resize(cv2.cvtColor(cv2.imread(modes[preds[i]], -1), cv2.COLOR_BGRA2RGBA), (box.right() - box.left(), box.bottom() - box.top()), interpolation = cv2.INTER_CUBIC)
    rows, cols, d = emoji.shape
    #plt.imshow(emoji)

    diag = np.sqrt((box.bottom() - box.top())**2 + (box.right() - box.left())**2)
    wexp = (diag - (box.right() - box.left()))/2
    hexp = (diag - (box.bottom() - box.top()))/2
    wexp = int(wexp)
    hexp = int(hexp)
    emoji = cv2.copyMakeBorder(emoji, hexp, hexp, wexp, wexp, cv2.BORDER_REPLICATE)
    #plt.imshow(emoji)

    src = np.array([(cols*(20.0/512.0) + wexp, rows*(200.0/512.0) + hexp), (cols*(256.0/512.0)+ wexp, rows*(495.0/512.0)+ hexp), (cols*(492.0/512.0)+ wexp, rows*(200.0/512.0)+ hexp)])
    #src = np.array([(20, 200), (256, 495), (492, 200)])
    src = np.uint8(src)
    src = np.float32(src)
    dest = np.array([(shapes.part(0).x - box.left()+ wexp, shapes.part(0).y - box.top()+ hexp),(shapes.part(8).x-box.left()+ wexp, shapes.part(8).y - box.top()+ hexp),(shapes.part(16).x - box.left()+ wexp, shapes.part(16).y - box.top()+ hexp)])
    dest = np.float32(dest)
    rows, cols, d = emoji.shape
    trans = cv2.getAffineTransform(src,dest)
    #plt.imshow(trans)
    #print trans
    emoji = cv2.warpAffine(emoji, trans, (cols, rows))
    i+=1


    #pint(emoji.shape)
    for c in range(0,3):
        img[box.top() - hexp: box.bottom() + hexp, box.left() - wexp: box.right() + wexp, c] = emoji[:,:,c] * (emoji[:,:,3]/255.0) + img[box.top() - hexp: box.bottom() + hexp, box.left() - wexp: box.right() + wexp, c] * (1.0 - emoji[:,:,3]/255.0)
    plt.imsave('result3.jpg',img)

        #cv2.imshow('frame',img)
        
    #end here
    
    # Our operations on the frame come here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame

    #cv2.imshow('frame',gray)
    #if cv2.waitKey(1) & 0xFF == ord('q'):
   #     break

#cap.release()
#cv2.destroyAllWindows()